# Structure Optimization

In [ ]:
%matplotlib notebook
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib import animation

import random
import time

In [ ]:
from potentials import gradients
from distances import vectors
from ewald_numpy import Ewald

In [ ]:
def grad_Ewald( x, q, boxsz=(0,1), pbc=False ):
    vecs = vectors(x, boxsz)
    E = Ewald(charges=q, boxsize=boxsz, alpha=5, r_cutoff=5, n_max=2, k_cutoff=5)
    return -E.force(x)

def grad_HLC( x, q, boxsize=(0,1), pbc=False ):
    """Sum of Harmonic, LJ and Coulombs gradients"""
    vecs = vectors(x, boxsize, pbc)
    return gradients.harmonic(x, boxsize) + gradients.LJ(vecs) - gradients.coulomb(vecs, q)
    
def descent( x, q, grad, a=1e-4, prec=1e-10, maxst=1e6, k=.1, boxsize=(0, 1), pbc=False ):
    """Gradient Descent
    
    Arguments:
        x    (float): position vectors (dim = n x 3)
        q: charge
        a    (float): 'learning rate' alpha = 1e-4
        prec (float): difference between steps, precision = 1e-10
        maxst  (int): max # of steps, maxst = 1e6
        k: factor harmonic pot
    
    Output:
        x: position array,
        step: # of steps needed to converge"""
    xmin, xmax = boxsize[0], boxsize[1]
    L = xmax - xmin
    x = x[None, :, :]
    step = 0
    f = grad(x[-1], q, pbc=pbc)
    x1 = x[-1] - a * f
    if pbc:
        x1 = x1 + (x1 < xmin) * L
        x1 = x1 - (x1 > xmax) * L  
    while step < maxst and np.linalg.norm(x[-1] - x1) > prec:
        x = np.append(x, x1[None, :, :], axis=0)
        f = grad(x[-1], q, pbc=pbc)
        x1 = x[-1] - a * f
        if pbc:
            x1 = x1 + (x1 < xmin) * L
            x1 = x1 - (x1 > xmax) * L
        step += 1
    return x, step

# Harmonic,  LJ & Coulomb
## 3D case

In [ ]:
def x_init_gen3D(N=3): 
    x = np.linspace(-N, N, N)
    XX, YY, ZZ = np.meshgrid(x,x,x)
    #q = np.array([[-1,1][random.randrange(2)] for i in range(N**3)])
    q = np.array([(-1)**j for i in range(N**2) for j in range(N)]) * 10
    x_init = np.array([XX.flatten(),YY.flatten(),ZZ.flatten()]).T
    x_init += np.random.uniform(low=-0.1, high=0.1, size=(N**3,3))
    #x_init *= 10
    assert len(x_init) == N**3
    return x_init, q

def init_x(N=3, dim=2, x_pert=0, x_ampl=1., q_ampl=1, alter=False):
    """Generate initial particle distribution and charges
        N**dim - # of particles
        dim - dimension, 2D or 3D
        x_pert - position perturbation, displaces particles
        x_fact - position factor, sizes the free space between particles
        q_ampl - amplification of charges
        alter - True: enable alternating charges; False: generate random charges
    
    Output:
        x: positions
        q: charges
    """
    import random
    # Positions
    distri = np.linspace(-N, N, N)
    if dim==2:
        XX, YY = np.meshgrid(distri, distri)
        x = np.array([XX.flatten(),YY.flatten()]).T
    elif dim==3:
        XX, YY, ZZ = np.meshgrid(distri, distri, distri)
        x = np.array([XX.flatten(),YY.flatten(),ZZ.flatten()]).T
    assert len(x) == N**dim
    x += np.random.uniform(low=-x_pert, high=x_pert, size=(N**dim,dim))
    x *= x_ampl
    # Charges
    if alter:
        #q = np.array([(-1)**(i+j) for i in range(N) for j in range(N**(dim-1))])
        q = np.array([(-1)**(i+j+k) for i in range(N) for j in range(N) for k in range(N**(dim-2))])
    else:
        q = np.array([[-1,1][random.randrange(2)] for i in range(N**dim)])
    q *= q_ampl
    return x, q


In [ ]:
x_init, q = x_init_gen3D(2)
print(x_init.shape, q)
x_init, q = init_x(N=2, dim=3, alter=True)
print(x_init.shape, q)
print(x_init.shape)
positions, nsteps = descent(x_init, q, grad_HLC, k=0.1, 
                       a=1e-4, prec=1e-6, maxst=80000)
print('# of steps:', nsteps, '| positions.shape:', positions.shape)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(9.5,9.5))
ax = fig.add_subplot(111, projection='3d')
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], x_init[:,2], c=q)
#circles = [plt.Circle(r, radius=0.5, fill=False) 
#            for i,r in enumerate(x_init)]
#for c in circles:
#    plt.gca().add_patch(c)
ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(-3, 3)
def animate(i):
    index = 10*i
    data = positions[index]
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    #for i, c in enumerate(circles):
    #    c.center = data[i]
    return scat,

#anim = animation.FuncAnimation(fig, animate, interval=1)
anim = animation.FuncAnimation(fig, animate, interval=20, frames=1050, repeat=False)